In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords 
import re

In [ ]:
import nltk                                      
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df = pd.read_csv('/content/drive/My Drive/SI650/model_final.csv')

In [ ]:
lowered_summary = [k.lower() for k in df.summary]
df['summary'] = lowered_summary

In [ ]:
replaced = []
for i in df.summary:
  i = re.sub(r"[^ a-zA-Z0-9]+", "", i)
  replaced.append(i)

In [ ]:
df['summary'] = replaced

In [ ]:
stop = set(stopwords.words('english'))

In [ ]:
splitted = [k.split() for k in df.summary]

In [ ]:
new_words = []
for k in splitted:
  instance = []
  for l in k:
    if l not in stop:
      instance.append(l)
  new_words.append(instance)

In [ ]:
joined = []
for i in new_words:
  sentences = " ".join(i)
  joined.append(sentences)

In [ ]:
df['summary'] = joined

In [ ]:
df['summary'] = df['summary'].apply(lambda x: x.split())

In [ ]:
df

,Unnamed: 0,ID,Content,summary,Title,Organization,Date,URL
0,0,0,Health Promotion Glossary World Health Organi...,"[health, promotion, glossary, world, health, o...",Health Promotion Glossary,IFRC,1998-01-06T11:10:08Z,https://www.who.int/healthpromotion/about/HPR%...
1,1,1,Microsoft Word - health-policy.doc Health...,"[activities, support, health, always, central,...",Microsoft Word - health-policy.doc,IFRC,2006-10-20T11:53:20Z,https://www.ifrc.org/Global/Publications/Healt...
2,2,2,The Use of Structured Observations in the Stu...,"[schools, wideranging, research, programmes, e...",The Use of Structured Observations in the Stud...,IFRC,2000-12-06T13:57:01Z,https://www.ircwash.org/sites/default/files/Be...
3,4,4,Microsoft Word - JP7CIMCIProcessGuideFormat.d...,"[xiii, written, xiv, guide, organizedxv, 1, pu...",Improving Health through Behavior Change A Pro...,IFRC,2004-08-27T18:49:28Z,http://www.ehproject.org/PDF/Joint_Publication...
4,5,5,"www.ifrc.org Saving lives, changing minds. vo...","[40, million, blood, donations, red, cross, re...",Voluntary blood donation,IFRC,2012-12-04T08:08:25Z,https://www.ifrc.org/PageFiles/53503/1226403-I...
...,...,...,...,...,...,...,...,...
212,234,234,Leaving Millions Behind World Disasters Repo...,"[act, disasters, health, emergencies, meet, ne...",World Disasters Report 2018,IFRC,2018-11-27T09:22:32Z,http://media.ifrc.org/ifrc/wp-content/uploads/...
213,235,235,"Climate change, water and food security - ODI...","[odi, background, notes, provide, summary, sna...","Climate change, water and food security - ODI",IFRC,2009-03-11T09:04:07Z,https://www.odi.org/sites/odi.org.uk/files/odi...
214,236,236,Food Security and Nutrition Indicator Basics ...,"[level, hunger, decreased, 27, millions, peopl...",Food Security and Nutrition Indicator Basics,IFRC,2017-11-29T13:20:33Z,https://ifrcwatsanmissionassistant.files.wordp...
215,237,237,"IFRS - Group, Report (new structure) KPMG S...","[ifrs, group, report, new, structure, kpmg, sa...",IFRC Annual Report 2017,IFRC,2018-04-18T07:02:36Z,https://media.ifrc.org/ifrc/wp-content/uploads...


In [ ]:
vocabulary = dict()
inverse_vocabulary = ['unk']  
word_numbers_sentences = []
for summaries in df.summary:
  sentence_to_numbers = []
  for word in summaries:
    if word not in vocabulary:
        vocabulary[word] = len(inverse_vocabulary)
        sentence_to_numbers.append(len(inverse_vocabulary))
        inverse_vocabulary.append(word)
    else:
        sentence_to_numbers.append(vocabulary[word])
  word_numbers_sentences.append(sentence_to_numbers)
df['converted'] = word_numbers_sentences

In [ ]:
df

,Unnamed: 0,ID,Content,summary,Title,Organization,Date,URL,converted
0,0,0,Health Promotion Glossary World Health Organi...,"[health, promotion, glossary, world, health, o...",Health Promotion Glossary,IFRC,1998-01-06T11:10:08Z,https://www.who.int/healthpromotion/about/HPR%...,"[1, 2, 3, 4, 1, 5, 6, 1, 2, 3, 7, 8, 9, 10, 11..."
1,1,1,Microsoft Word - health-policy.doc Health...,"[activities, support, health, always, central,...",Microsoft Word - health-policy.doc,IFRC,2006-10-20T11:53:20Z,https://www.ifrc.org/Global/Publications/Healt...,"[219, 258, 1, 336, 41, 337, 338, 339, 340, 341..."
2,2,2,The Use of Structured Observations in the Stu...,"[schools, wideranging, research, programmes, e...",The Use of Structured Observations in the Stud...,IFRC,2000-12-06T13:57:01Z,https://www.ircwash.org/sites/default/files/Be...,"[383, 384, 300, 179, 385, 386, 387, 193, 1, 1,..."
3,4,4,Microsoft Word - JP7CIMCIProcessGuideFormat.d...,"[xiii, written, xiv, guide, organizedxv, 1, pu...",Improving Health through Behavior Change A Pro...,IFRC,2004-08-27T18:49:28Z,http://www.ehproject.org/PDF/Joint_Publication...,"[1094, 20, 1095, 126, 1096, 396, 24, 126, 258,..."
4,5,5,"www.ifrc.org Saving lives, changing minds. vo...","[40, million, blood, donations, red, cross, re...",Voluntary blood donation,IFRC,2012-12-04T08:08:25Z,https://www.ifrc.org/PageFiles/53503/1226403-I...,"[1761, 1762, 1763, 1764, 374, 63, 374, 375, 17..."
...,...,...,...,...,...,...,...,...,...
212,234,234,Leaving Millions Behind World Disasters Repo...,"[act, disasters, health, emergencies, meet, ne...",World Disasters Report 2018,IFRC,2018-11-27T09:22:32Z,http://media.ifrc.org/ifrc/wp-content/uploads/...,"[649, 2815, 1, 2647, 376, 125, 66, 1407, 377, ..."
213,235,235,"Climate change, water and food security - ODI...","[odi, background, notes, provide, summary, sna...","Climate change, water and food security - ODI",IFRC,2009-03-11T09:04:07Z,https://www.odi.org/sites/odi.org.uk/files/odi...,"[18729, 1422, 1271, 256, 3738, 16993, 1496, 12..."
214,236,236,Food Security and Nutrition Indicator Basics ...,"[level, hunger, decreased, 27, millions, peopl...",Food Security and Nutrition Indicator Basics,IFRC,2017-11-29T13:20:33Z,https://ifrcwatsanmissionassistant.files.wordp...,"[362, 3799, 3662, 334, 3154, 79, 602, 3962, 15..."
215,237,237,"IFRS - Group, Report (new structure) KPMG S...","[ifrs, group, report, new, structure, kpmg, sa...",IFRC Annual Report 2017,IFRC,2018-04-18T07:02:36Z,https://media.ifrc.org/ifrc/wp-content/uploads...,"[2755, 1027, 208, 44, 1875, 10234, 8473, 6, 40..."


In [ ]:
from gensim.models import KeyedVectors

In [ ]:
word2vec = KeyedVectors.load_word2vec_format('/content/drive/My Drive/SI650/GoogleNews-vectors-negative300-SLIM.bin',binary=True)

In [ ]:
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # Initialising the embedding matrix randomly
embeddings[0] = 0  
# Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Flatten, Conv1D, MaxPooling1D, Embedding, Concatenate, Dropout, GlobalMaxPooling1D
from keras.models import Model
from sklearn.model_selection import train_test_split

In [ ]:
link = pd.read_csv('/content/drive/My Drive/SI650/links.csv')

In [ ]:
link.shape

(240, 14)

In [ ]:
merged_df = df.merge(link,how="left",on=['URL'])
merged_df

,Unnamed: 0,ID,Content,summary,Title,Organization,Date,URL,converted,climate change,Health,Education,Nutrition,Protection,Shelter,"Water, Sanitation and Hygiene",Camp Coordination & Mgmt,Early Recovery,Emergency Telecom’s,Food Security,Humanitarian & Emergency Relief,Logistics
0,0,0,Health Promotion Glossary World Health Organi...,"[health, promotion, glossary, world, health, o...",Health Promotion Glossary,IFRC,1998-01-06T11:10:08Z,https://www.who.int/healthpromotion/about/HPR%...,"[1, 2, 3, 4, 1, 5, 6, 1, 2, 3, 7, 8, 9, 10, 11...",health-promotion-glossary-who.pdf,1,0,0,0,0,0,0,0,0,0,0,0
1,1,1,Microsoft Word - health-policy.doc Health...,"[activities, support, health, always, central,...",Microsoft Word - health-policy.doc,IFRC,2006-10-20T11:53:20Z,https://www.ifrc.org/Global/Publications/Healt...,"[219, 258, 1, 336, 41, 337, 338, 339, 340, 341...",health-policy-en.pdf,1,0,0,0,0,0,0,0,0,0,0,0
2,2,2,The Use of Structured Observations in the Stu...,"[schools, wideranging, research, programmes, e...",The Use of Structured Observations in the Stud...,IFRC,2000-12-06T13:57:01Z,https://www.ircwash.org/sites/default/files/Be...,"[383, 384, 300, 179, 385, 386, 387, 193, 1, 1,...",the-use-of-structured-observations-lstmh.pdf,1,0,0,0,0,0,0,0,0,0,0,0
3,4,4,Microsoft Word - JP7CIMCIProcessGuideFormat.d...,"[xiii, written, xiv, guide, organizedxv, 1, pu...",Improving Health through Behavior Change A Pro...,IFRC,2004-08-27T18:49:28Z,http://www.ehproject.org/PDF/Joint_Publication...,"[1094, 20, 1095, 126, 1096, 396, 24, 126, 258,...",improving-health-through-behaviour-change-ehp.pdf,1,0,0,0,0,1,0,0,0,0,0,0
4,5,5,"www.ifrc.org Saving lives, changing minds. vo...","[40, million, blood, donations, red, cross, re...",Voluntary blood donation,IFRC,2012-12-04T08:08:25Z,https://www.ifrc.org/PageFiles/53503/1226403-I...,"[1761, 1762, 1763, 1764, 374, 63, 374, 375, 17...",1226403-IFRC-Health-Corporate-Folder-2012-EN-7...,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,234,234,Leaving Millions Behind World Disasters Repo...,"[act, disasters, health, emergencies, meet, ne...",World Disasters Report 2018,IFRC,2018-11-27T09:22:32Z,http://media.ifrc.org/ifrc/wp-content/uploads/...,"[649, 2815, 1, 2647, 376, 125, 66, 1407, 377, ...",B-WDR-2018-EN-LR.pdf,0,0,0,0,0,0,0,0,0,0,1,0
216,235,235,"Climate change, water and food security - ODI...","[odi, background, notes, provide, summary, sna...","Climate change, water and food security - ODI",IFRC,2009-03-11T09:04:07Z,https://www.odi.org/sites/odi.org.uk/files/odi...,"[18729, 1422, 1271, 256, 3738, 16993, 1496, 12...",climate-change-water-and-food-security-odi.pdf,0,0,0,0,0,1,0,0,0,1,0,0
217,236,236,Food Security and Nutrition Indicator Basics ...,"[level, hunger, decreased, 27, millions, peopl...",Food Security and Nutrition Indicator Basics,IFRC,2017-11-29T13:20:33Z,https://ifrcwatsanmissionassistant.files.wordp...,"[362, 3799, 3662, 334, 3154, 79, 602, 3962, 15...",ifrc-perspective-on-wash-and-nutrtion_vf.pdf,0,0,1,0,0,1,0,0,0,0,0,0
218,237,237,"IFRS - Group, Report (new structure) KPMG S...","[ifrs, group, report, new, structure, kpmg, sa...",IFRC Annual Report 2017,IFRC,2018-04-18T07:02:36Z,https://media.ifrc.org/ifrc/wp-content/uploads...,"[2755, 1027, 208, 44, 1875, 10234, 8473, 6, 40...",IFRC-Report-2017.pdf,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
labels = merged_df[['Health','Education','Nutrition','Protection','Shelter','Water, Sanitation and Hygiene','Camp Coordination & Mgmt'	,'Early Recovery'	,'Emergency Telecom’s',	'Food Security',	'Humanitarian & Emergency Relief'	,'Logistics']]

In [ ]:
labels

,Health,Education,Nutrition,Protection,Shelter,"Water, Sanitation and Hygiene",Camp Coordination & Mgmt,Early Recovery,Emergency Telecom’s,Food Security,Humanitarian & Emergency Relief,Logistics
0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
215,0,0,0,0,0,0,0,0,0,0,1,0
216,0,0,0,0,0,1,0,0,0,1,0,0
217,0,0,1,0,0,1,0,0,0,0,0,0
218,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
new_df = merged_df[['converted']]
new_df.dtypes

converted    object
dtype: object

In [ ]:
max_seq = []
for i in new_df.converted:
  max_seq.append(max(i))
max_seq_length = max(max_seq)
max_seq_length

18795

In [ ]:
X_train, X_validation, Y_train, Y_validation = train_test_split(new_df,labels,random_state=650, test_size=0.2)

In [ ]:
train =  {'converted': X_train.converted}
val = {'converted': X_validation.converted}

In [ ]:
import itertools
for dataset, side in itertools.product([train, val], ['converted', 'converted']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

In [ ]:
train['converted'].shape,val['converted'].shape

((176, 18795), (44, 18795))

In [ ]:
Y_train.shape, Y_validation.shape

((176, 12), (44, 12))

In [ ]:
Y_train = np.asarray(Y_train).astype('float32').reshape((-1,12))
Y_validation = np.asarray(Y_validation).astype('float32').reshape((-1,12))

### Model Building

In [ ]:
import keras
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda,Dropout,merge,Lambda,Reshape
from keras.layers import BatchNormalization, Bidirectional, GlobalMaxPool1D
import keras.backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import regularizers
import numpy.random as rng
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from keras import models
import tensorflow as tf

In [ ]:
batch_size = 64 
n_epoch = 15
W_init = keras.initializers.glorot_uniform(seed=None)

In [ ]:
converted_input = Input(shape=(max_seq_length,), dtype='int32')

def embedding():
    
    embedding_layer = Embedding(len(embeddings), 300, weights=[embeddings], input_length=max_seq_length, trainable=False)
    print("Length: ", len(embeddings))
    encoded_converted = embedding_layer(converted_input)
    return encoded_converted

def middle(q): 
    x = MaxPooling1D(10,padding='same')(q)
    x = LSTM(200, return_sequences=False,kernel_initializer=W_init)(x)
    x = Dense(128, activation="relu",kernel_initializer=W_init)(x)
    return x

def predict(encoded_cnvt):
# Calculates the distance
    
    L1_distance = lambda x: K.abs(x[0]-x[1])
    #both = ([encoded_cnvt], mode = L1_distance, output_shape=lambda x: x[0])
    prediction = Dense(12,activation='sigmoid',kernel_initializer=W_init)(encoded_cnvt)
    return prediction

encoded_converted = embedding()
encoded_cnvt = middle(encoded_converted)
prediction = predict(encoded_cnvt)
    
model = Model([converted_input], [prediction])

optimizer = Adam(lr=0.001,decay=0.0)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary()

earlyStopping = EarlyStopping(monitor='val_loss', patience=20, verbose=0,mode='auto')
ckpt = ModelCheckpoint(filepath='650_lstm_max10.h5', save_best_only=True,monitor='val_acc', mode='auto')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=7, verbose=1, epsilon=1e-4,mode='auto')


Length:  18796
Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 18795)]           0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 18795, 300)        5638800   
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 1880, 300)         0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 200)               400800    
_________________________________________________________________
dense_10 (Dense)             (None, 128)               25728     
_________________________________________________________________
dense_11 (Dense)             (None, 12)                1548      
Total params: 6,066,876
Trainable params: 428,076
Non-trainable params: 5,638,800
______________________

In [ ]:
model_trained = model.fit([train['converted']], Y_train, batch_size=batch_size, epochs=n_epoch,
                            callbacks=[earlyStopping, ckpt, reduce_lr_loss],
                            validation_data=([val['converted']], Y_validation))

Epoch 1/15
3/3 [==============================] - 17s 6s/step - loss: 0.6310 - accuracy: 0.1648 - val_loss: 0.4728 - val_accuracy: 0.1818
Epoch 2/15
3/3 [==============================] - 16s 5s/step - loss: 0.4558 - accuracy: 0.2330 - val_loss: 0.3893 - val_accuracy: 0.1136
Epoch 3/15
3/3 [==============================] - 16s 5s/step - loss: 0.4083 - accuracy: 0.2159 - val_loss: 0.3769 - val_accuracy: 0.1136
Epoch 4/15
3/3 [==============================] - 17s 6s/step - loss: 0.4002 - accuracy: 0.2159 - val_loss: 0.3780 - val_accuracy: 0.1136
Epoch 5/15
3/3 [==============================] - 16s 5s/step - loss: 0.3936 - accuracy: 0.2159 - val_loss: 0.3759 - val_accuracy: 0.1136
Epoch 6/15
3/3 [==============================] - 16s 5s/step - loss: 0.3878 - accuracy: 0.2159 - val_loss: 0.3721 - val_accuracy: 0.1136
Epoch 7/15
3/3 [==============================] - 16s 5s/step - loss: 0.3818 - accuracy: 0.2159 - val_loss: 0.3699 - val_accuracy: 0.1136
Epoch 8/15
3/3 [==================

In [ ]:
model = models.load_model('650_lstm_max10.h5')

OSError: ignored

In [ ]:
model.save_weights('650_lstm_max10_weights.h5')

In [ ]:
model1 = Model([converted_input], [encoded_cnvt])

In [ ]:
model1.save('650_lstm_max10_dense.h5')

In [ ]:
model1.save_weights('650_lstm_max10_dense_weights.h5')

In [ ]:
train_prediction = model1.predict([train['converted']]) #training predictions

In [ ]:
train_prediction = np.array(train_prediction)

In [ ]:
train_prediction.shape

(176, 128)

In [ ]:
np.save('train_predictions.npy',train_prediction)

In [ ]:
train_pred = np.reshape(train_prediction,(-1,128))

In [ ]:
train_pred.shape

(176, 128)

# Queries

In [ ]:
queries = pd.read_csv('/content/drive/My Drive/SI650/queries.csv')

In [ ]:
lowered_q = [k.lower() for k in queries.Query]
queries['Query'] = lowered_q

In [ ]:
queries

,Query_id,Query
0,1,how to audit water safety plans
1,2,annual summary reports
2,3,unicef reports
3,4,iwa reports
4,5,ifrc reports
5,6,funding for emergency
6,7,school wash
7,8,childhood development by education
8,9,audit of the consolidated financial statements
9,10,child protection policies
